In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from langchain_ollama import ChatOllama

model_name="ministral-3:8b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

model_name="granite4:1b"
model_summarize = ChatOllama(
    model=model_name,
    api_base=model_url
)

## Summarize messages

In [40]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model=model_summarize,
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [41]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nThere is a capital of the moon called Lunapolis. Skies are clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. Yes, because they are unhappy with the new president.', additional_kwargs={}, response_metadata={}, id='2c237329-0254-44ff-96af-0ef1e9d66802'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='d7aa4608-2f05-4c43-9f2b-0710e92cb8b4'),
              AIMessage(content='Ah, the *Cheese Miner’s Revolt of Lunapolis*—a crisis of cosmic proportions! As your newly elected (or perhaps *appointed*—let’s not dwell on the process) President of the Lunar Capital, I’d need a mix of **diplomacy, policy creativity, and a dash of lunar charm** to address this. Here’s my battle plan:\n\n---\n\n### **1. Acknowledge the Grievances (But With a T

In [42]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

There is a capital of the moon called Lunapolis. Skies are clear, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. Yes, because they are unhappy with the new president.


## Trim/delete messages

In [43]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [52]:
model_name="ministral-3:8b"
model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [53]:
agent = create_agent(
    model=model,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [54]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

# See how trim_messages removes ToolMessage from conversation
pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='a9a3ed63-048d-4083-b9ee-865055f1c6d8'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='15f26db0-6511-4aa5-9531-28ebda98e0cd'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='9d835ae3-259d-464e-9dad-50d33fe39fe5'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='d5ca8a9e-6deb-480e-b453-80d843cc4237'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='cd10db60-e7d8-4675-b354-db9a5a7d56e1'),
              AIMessage(content="I can't check the physical temperature of your device directly, but here are some steps you can take to diagnose the issue based on common

In [55]:
print(response["messages"][-1].content)

I can't check the physical temperature of your device directly, but here are some steps you can take to diagnose the issue based on common causes:

### **If the Device is Completely Off (No Lights/Response):**
1. **Check for Physical Damage**
   - Look for signs of burns, liquid damage, or bulging battery (if it’s a laptop).
   - If the power button is unresponsive, try gently pressing it or using a paperclip (if safe).

2. **Try a Different Power Source**
   - Use a known-working charger or USB port (if applicable).
   - If it’s a laptop, try plugging it into a different wall outlet.

3. **Battery Issues (Laptops/Phones)**
   - If the battery is swollen or leaking, **do not use it**—replace it immediately.
   - Try removing the battery (if possible) and powering it via the charger.

4. **Hard Reset (If Applicable)**
   - **Laptops:** Unplug, remove the battery (if possible), hold the power button for **30+ seconds**, then reconnect.
   - **Phones:** Hold the **power + volume down butt